In [49]:
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.datasets import imdb 
from sklearn.metrics import classification_report

In [26]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, Layer, Input
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [19]:
from tensorflow.keras import models,layers,activations,losses,optimizers,metrics

In [2]:
n_unique_words = 10000
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=n_unique_words)

In [3]:
maxlen = 200
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [67]:
#pure bilstm
model = Sequential()
model.add(Embedding(n_unique_words, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 bidirectional_12 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [69]:
#model.fit(x_train, y_train, validation_split = 0.2,epochs = 12, verbose = True,batch_size = 64)

In [36]:
class Attention(Layer):
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(Attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()

In [57]:
inp = Input(shape=(200, ))
x = Embedding(10000, 128, trainable=False)(inp)
conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(x)
maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)
conv_2 = Conv1D(32, 7, activation='relu', name='conv1d_2')(dropout_1)
maxpool_2 = MaxPooling1D(8,padding='same', name='maxpool1d_2')(conv_2)
dropout_2 = Dropout(0.2, name='dropout_2')(maxpool_2)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(dropout_2)
x = Attention()(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_13 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 192, 32)           36896     
                                                                 
 maxpool1d_1 (MaxPooling1D)  (None, 12, 32)            0         
                                                                 
 dropout_1 (Dropout)         (None, 12, 32)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 6, 32)             7200      
                                                                 
 maxpool1d_2 (MaxPooling1D)  (None, 1, 32)             0   

In [58]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
file_path = 'model.hdf5'
ckpt = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=1)

In [72]:
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.2)

Epoch 1/5
157/157 [==============================] - 183s 1s/step - loss: 0.1642 - accuracy: 0.9427 - val_loss: 0.3367 - val_accuracy: 0.8734
Epoch 2/5
157/157 [==============================] - 184s 1s/step - loss: 0.1197 - accuracy: 0.9603 - val_loss: 0.4186 - val_accuracy: 0.8708
Epoch 3/5
157/157 [==============================] - 182s 1s/step - loss: 0.0963 - accuracy: 0.9675 - val_loss: 0.4396 - val_accuracy: 0.8674
Epoch 4/5
157/157 [==============================] - 183s 1s/step - loss: 0.0813 - accuracy: 0.9748 - val_loss: 0.4665 - val_accuracy: 0.8446
Epoch 5/5
157/157 [==============================] - 181s 1s/step - loss: 0.0684 - accuracy: 0.9779 - val_loss: 0.4829 - val_accuracy: 0.8544


In [ ]:
y_pred = model.predict(x_test)

350/782 [============>.................] - ETA: 17s

In [61]:
y_pred.shape

(25000, 1)

In [62]:
y_test.shape

(25000,)

In [65]:
y_pred_p = (y_pred > 0.5).astype('int64')

In [66]:
print(classification_report(y_test, y_pred_p))

              precision    recall  f1-score   support

           0       0.76      0.80      0.78     12500
           1       0.79      0.75      0.77     12500

    accuracy                           0.77     25000
   macro avg       0.77      0.77      0.77     25000
weighted avg       0.77      0.77      0.77     25000

